# Regressão logística

Nesta aula, iremos trabalhar com o DataSet do Titanic.

Vamos tentar montar um algoritmo de classificação, Sobrevivente ou Falecido, utilizando a regressão logística no Python.

Usaremos uma versão "semi-limpa" do DataSet do Titanic.

## Importar bibliotecas
Vamos importar algumas bibliotecas para começar!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Os dados

Leitura e importação do ficheiro titanic_train.csv para um pandas DataFrame.

In [ ]:
train = pd.read_csv('titanic_train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

# Análise exploratória dos dados

Vamos começar por efetuar uma análise exploratória de dados, verificando os dados que faltam! Por exemplo, no campo "Cabin" existem dados NaN, ou seja, não se conhece o valor.

## Dados ausentes

Podemos usar a seaborn para criar um mapa de calor (HEATMAP - https://seaborn.pydata.org/generated/seaborn.heatmap.html) simples para visualizar a perda de dados!

O método isnull() do pandas, irá retornar True quando o valor é Null e False quando não é Null.

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(train.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

Constata-se que existem dados em falta, relativamente a Age, Cabin e Embarked. A proporção de Age em falta é pequena, contudo, perdemos muitos dados relativamente a Cabin.

In [ ]:
# Podemos usar a seaborn para criar um COUNTPLOT - https://seaborn.pydata.org/generated/seaborn.countplot.html?highlight=countplot

#O COUNTPLOT irá mostrar a contagem das observações para a categoria Survived utilizando barras.

In [ ]:
sns.countplot(x = 'Survived', data = train)

In [ ]:
sns.countplot(x = 'Survived', hue = 'Sex', data = train)

In [ ]:
sns.countplot(x = 'Survived', hue = 'Pclass', data = train)

In [ ]:
plt.hist(train['Age'], bins=30)
plt.xlabel("Age")

In [ ]:
sns.countplot(x = 'SibSp', data = train)

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train['Fare'], bins=50)
plt.xlabel("Fare")

___
## Limpar os dados

Queremos preencher os dados das idades em falta, em vez de simplesmente apagar as linhas de dados das idades que faltam. 
Uma maneira de fazer isto, será preencher com a idade média de todos os passageiros.
No entanto, podemos ser mais inteligentes e verificar a idade média através da classe de passageiros. Por exemplo:


In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x = 'Pclass', y = 'Age', data = train)

Podemos verificar que os passageiros mais ricos (que estão nas classes superiores) tendem a ser mais velhos, o que faz sentido. Usaremos os valores da idade média para preenchimento de Age, com base em Pclass.

In [ ]:
def input_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

Aplicação da função!

Método apply() - https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.apply.html

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(input_age, axis = 1)

Verificação do HEATMAP

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Apagar a coluna Cabin e a linha em Embarked.

In [ ]:
del train['Cabin']

In [ ]:
train.head()

Quais os registos incompletos?

In [ ]:
train[train.isna().any(axis=1)]

O Método dropna apaga linhas que contêm qualquer tipo de informação falsa.

In [ ]:
train.dropna(inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

## Converter recursos categóricos

Existe a necessidade de converter os dados categóricos em variáveis dummy, usando o pandas! Caso contrário, o algoritmo de Machine Learning não será capaz de aceitar estas variáveis como entradas, diretamente.

In [ ]:
train.info()

O Método get_dummies() irá converter para números os dados categóricos de determinadas variáveis, nomeadamente Sex e Embarked.
O parâmetro drop_first irá eliminar a primeira coluna, para não criar constrangimentos ao modelo.

In [ ]:
pd.get_dummies(train['Sex'])

In [ ]:
sex = pd.get_dummies(train['Sex'], drop_first = True)
embark = pd.get_dummies(train['Embarked'], drop_first = True)

In [ ]:
train.drop(['Sex', 'PassengerId','Embarked','Name','Ticket'], axis = 1, inplace = True)

In [ ]:
train = pd.concat([train, sex, embark], axis = 1)

In [ ]:
train.head()

Os dados estão prontos para o modelo

# Construir o modelo de Regressão Logística

Vamos começar por dividir os dados num conjunto de treino e num conjunto de testes (existe outro ficheiro test.csv que pode ser utilizado, caso se pretendam utilizar todos os dados).

## Divisão treino-teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S']]
Y = train['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

## Treino e Previsão

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# logmodel = LogisticRegression()
logmodel = LogisticRegression(max_iter=1000)
#logmodel = LogisticRegression(random_state=0, multi_class='ovr', penalty='l2', solver='liblinear', max_iter=1000)

Solução do erro no fit - 
https://www.codegrepper.com/code-examples/python/how+to+increase+max_iter+in+logistic+regression

In [ ]:
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

Vamos seguir em frente para avaliar o modelo!

## Avaliação

Podemos verificar a precisão usando o relatório de classificação! Ou então a Matriz, tal como nos slides da aula.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
mat = confusion_matrix(y_test,predictions)
tx_ok=mat[0][0]+mat[1][1]
tx_nok=mat[1][0]+mat[0][1]
print("Tx sucesso:", tx_ok/(tx_ok+tx_nok), ", Tx insucesso:", tx_nok/(tx_ok+tx_nok))